In [22]:
import bz2
import pandas as pd
import os
import shutil

from torch.utils.data import Dataset, DataLoader

import nltk
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from torchtext.vocab import build_vocab_from_iterator

import matplotlib.pyplot as plt
import seaborn as sns
import json
import random
from collections import Counter
import numpy as np
from torch.nn.utils.rnn import pad_sequence
import torch
from torch import nn

from torch.utils.data import TensorDataset, DataLoader

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn import metrics
import gc

import sys
sys.path.append('../')

import contractions 

import re
import gc
import pickle


#The essentials
STOPWORDS = set(stopwords.words("english"))
print('stopwords = ', STOPWORDS)

import bz2

stopwords =  {'why', "didn't", 'been', 'do', 'll', 'being', 'them', 'under', 'further', 'just', 'shan', 'through', 'its', 'themselves', 'should', 'some', 'our', 'by', 'over', 'about', 'ours', 'doing', 'these', 'but', 'i', 'during', 'between', 'didn', 'd', "she's", "aren't", 'what', 'did', 'before', 'couldn', 'on', 'than', 'isn', 'it', 'ain', 'was', 'out', 'both', 'in', 'their', 'again', 'hadn', 'we', 'her', 'hasn', 'here', 'or', 'while', 'myself', 'once', 'off', 'his', 'those', "you'll", "it's", 'mightn', 'won', 't', 'that', "should've", 'y', 'ourselves', "won't", 's', "isn't", "that'll", 'an', 'hers', "shan't", 'all', 'm', 'at', 'because', 'which', "shouldn't", 'up', 'how', 'himself', 'when', 'until', 'weren', 'does', 'nor', 'are', 'there', 'very', 'not', 'same', 'itself', 'shouldn', 'only', 'other', 've', 'has', 'doesn', 'haven', 'below', 'now', "you've", 'to', 'most', 'ma', 'down', 'having', 'can', 'who', 'after', 'yours', 'each', 'o', 'so', 'him', 'wasn', 'they', 'too', 'he', 'and'

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jamesnguyen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Load, clean, and process  data

In [11]:
root = os.getcwd()
datasetpath = os.path.join(root, 'dataset')
assert os.path.exists(datasetpath), f'dataset path does not exist {datasetpath}'

In [12]:
train_txt = bz2.BZ2File(os.path.join(datasetpath, 'train.ft.txt.bz2'))
# test_txt = bz2.BZ2File(os.path.join(datasetpath, 'test.ft.txt.bz2'))

In [13]:
train_lines = train_txt.readlines()
del train_txt
gc.collect()
# test_file = test_txteadlines()

488

In [14]:
for line in train_lines[:10]:
    print(line)

b'__label__2 Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^\n'
b"__label__2 The best soundtrack ever to anything.: I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This in my opinino is Yasunori Mitsuda's ultimate masterpiece. The music is timeless and I'm been listening to it for years now and its beauty simply refuses to fade.The price tag on this is pretty staggering I must say, but if you are going to buy any cd for this much money, this is the only one that I feel would be worth every penny.\n"
b'__label__2 Amazing!: This sou

Convert bytes object into utf-8

In [16]:
#only tkae 10% of this dataset
LIMIT = int(0.05 * len(train_lines))
print('limit = ', LIMIT)

truncated_train_lines = train_lines[:LIMIT]

TRAIN_RATIO = 0.8
train_len = int(len(truncated_train_lines) * TRAIN_RATIO)

train_array = [x.decode('utf-8') for x in truncated_train_lines[:train_len]]
val_array = [x.decode('utf-8') for x in truncated_train_lines[train_len:]]

#apply limit to a big dataset
train_array = train_array[:LIMIT]

print('train len = ', len(train_array))
print('val len = ', len(val_array))

limit =  180000
train len =  144000
val len =  36000


In [23]:
print(type(train_array))
for line in train_array[:10]:
    print(line)

save_path = os.path.join(datasetpath, 'truncated_train.ft.txt.bz2')
with bz2.open(save_path, 'wb') as file:
    file.write(bz2.compress(pickle.dumps(train_array)))

<class 'list'>
__label__2 Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^

__label__2 The best soundtrack ever to anything.: I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This in my opinino is Yasunori Mitsuda's ultimate masterpiece. The music is timeless and I'm been listening to it for years now and its beauty simply refuses to fade.The price tag on this is pretty staggering I must say, but if you are going to buy any cd for this much money, this is the only one that I feel would be worth every penny.

__label__2 Amazing!: Thi

In [8]:
def remove_urls(text):
    #if there's link in text, like www.something.com, https://www.something.com, 
    # replace it with the <url> token
    if 'www.' in text or 'http:' in text or 'https:' in text or '.com' in text:
        text = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", text)
    return text

def remove_digits(text):
    return re.sub("\d", '', text)

def remove_punctation(text):
    return re.sub(r'[^\w\s]',' ',text)

def expand_contraction(text):
    return contractions.fix(text)

def remove_stopwords(text):
    return ' '.join([word for word in text.split(' ') if word not in STOPWORDS])

def process_line(line):
    '''
    extract feature and label from line and process the text
    @params: 
        line: string, format: __label__2 some text.
    @return:
        feature: string
        label: int, 0: bad review, 1 good review
    '''
    #Each line has format: __label__2 some text. 
    #The first part is label, the rest is text feature
    parts = line.split()
    label = parts[0]
    feature = ' '.join(parts[1:])
    #lower case the features
    feature = feature.lower()
    #remove urls in text
    feature = remove_urls(feature)
    #expand contractions
    feature = expand_contraction(feature) 
    #remove punctuations
    feature = remove_punctation(feature)
    #remove digits
    feature = remove_digits(feature)
    #remove stop words
    feature = remove_stopwords(feature)
    #tokenize
    tokens = nltk.word_tokenize(feature)

    #__label__1 = 0: bad review, __label__2 = 1,good review  
    return tokens, 0 if label == '__label__1' else 1 

# Extracting labels from sentences
X_train = []
y_train = []

for line in tqdm(train_array):
    feature, label = process_line(line)
    X_train.append(feature)
    y_train.append(label)

print(X_train[:10])
print('X train len = ', len(X_train))
print(y_train[:10])
print('y train len = ', len(y_train))


100%|██████████| 288000/288000 [02:40<00:00, 1796.37it/s]

[['stuning', 'even', 'for', 'the', 'non-gamer', ':', 'this', 'sound', 'track', 'was', 'beautiful', '!', 'it', 'paints', 'the', 'senery', 'in', 'your', 'mind', 'so', 'well', 'i', 'would', 'recomend', 'it', 'even', 'to', 'people', 'who', 'hate', 'vid', '.', 'game', 'music', '!', 'i', 'have', 'played', 'the', 'game', 'chrono', 'cross', 'but', 'out', 'of', 'all', 'of', 'the', 'games', 'i', 'have', 'ever', 'played', 'it', 'has', 'the', 'best', 'music', '!', 'it', 'backs', 'away', 'from', 'crude', 'keyboarding', 'and', 'takes', 'a', 'fresher', 'step', 'with', 'grate', 'guitars', 'and', 'soulful', 'orchestras', '.', 'it', 'would', 'impress', 'anyone', 'who', 'cares', 'to', 'listen', '!', '^_^'], ['the', 'best', 'soundtrack', 'ever', 'to', 'anything', '.', ':', 'i', "'m", 'reading', 'a', 'lot', 'of', 'reviews', 'saying', 'that', 'this', 'is', 'the', 'best', "'game", 'soundtrack', "'", 'and', 'i', 'figured', 'that', 'i', "'d", 'write', 'a', 'review', 'to', 'disagree', 'a', 'bit', '.', 'this', '

In [9]:
#save checkpoint np
X_TRAIN_OUTPUT_PATH = os.path.join(root, 'output', 'X_train.npy')
Y_TRAIN_OUTPUT_PATH = os.path.join(root, 'output', 'y_train.npy')
with open(X_TRAIN_OUTPUT_PATH, 'wb') as file:
    np.save(file, X_train)
with open(Y_TRAIN_OUTPUT_PATH, 'wb') as file:
    np.save(file, y_train)

c:\Users\nguye\anaconda3\envs\torch\lib\site-packages\numpy\lib\npyio.py:521: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


# Tokenize feature and build vocab

In [10]:
#load checkpoint np
#from this point down is for training. Only loading this part will relieve the memory
X_TRAIN_OUTPUT_PATH = os.path.join(root, 'output', 'X_train.npy')
Y_TRAIN_OUTPUT_PATH = os.path.join(root, 'output', 'y_train.npy')

with open(X_TRAIN_OUTPUT_PATH, 'rb') as file:
    X_train = np.load(file, allow_pickle=True)
with open(Y_TRAIN_OUTPUT_PATH, 'rb') as file:
    y_train = np.load(file, allow_pickle=True)

In [11]:
print(X_train[0])
print(type(X_train[0]))
print(type(X_train[0][0]))

['stuning', 'even', 'for', 'the', 'non-gamer', ':', 'this', 'sound', 'track', 'was', 'beautiful', '!', 'it', 'paints', 'the', 'senery', 'in', 'your', 'mind', 'so', 'well', 'i', 'would', 'recomend', 'it', 'even', 'to', 'people', 'who', 'hate', 'vid', '.', 'game', 'music', '!', 'i', 'have', 'played', 'the', 'game', 'chrono', 'cross', 'but', 'out', 'of', 'all', 'of', 'the', 'games', 'i', 'have', 'ever', 'played', 'it', 'has', 'the', 'best', 'music', '!', 'it', 'backs', 'away', 'from', 'crude', 'keyboarding', 'and', 'takes', 'a', 'fresher', 'step', 'with', 'grate', 'guitars', 'and', 'soulful', 'orchestras', '.', 'it', 'would', 'impress', 'anyone', 'who', 'cares', 'to', 'listen', '!', '^_^']
<class 'list'>
<class 'str'>


In [12]:
corpus = Counter() 
for i, tokens in enumerate(tqdm(X_train)):
    #inplace update because of large dataset
    # tokens = nltk.word_tokenize(feature)
    corpus.update(tokens)


100%|██████████| 288000/288000 [00:03<00:00, 83167.20it/s]


In [13]:
# Buiding vocab
print('len corpus = ', len(corpus))
print(corpus.most_common(10))

tokens = list(map(lambda x: x[0], corpus.most_common()))
print(tokens[:10])

tokens = ['<pad>', '<unk>'] + tokens

word2idx = {}
idx2word = {}

for index, token in enumerate(tqdm(tokens)):
    word2idx[token] = index
    idx2word[index] = token

print("vocab len = ", len(word2idx))

gc.collect()

len corpus =  343114
[('.', 1186413), ('the', 1174244), (',', 859565), ('i', 647376), ('and', 628593), ('a', 590073), ('to', 555912), ('it', 509192), ('of', 485206), ('this', 435750)]
['.', 'the', ',', 'i', 'and', 'a', 'to', 'it', 'of', 'this']


100%|██████████| 343116/343116 [00:00<00:00, 1584499.55it/s]


vocab len =  343116


9

In [14]:
#For debugging purposes
with open('vocab.txt', 'w') as file:
    json.dump(word2idx, file)


In [15]:
#Converting tokens to idx
# X_train_test = [None for _ in range(100)]

for i, tokens in enumerate(tqdm(X_train)):
    try:
        ids = []
        for token in tokens:
            #if token not found in vocab. put it in unk
            # id = word2idx['<unk>']
            id = 0
            if token in word2idx:
                id = word2idx[token]
            ids.append(id)

        #inplace because of large file
        X_train[i] = torch.tensor(ids)
    except Exception as e:
        print("Error occured", e)
        break

  0%|          | 0/288000 [00:00<?, ?it/s]

100%|██████████| 288000/288000 [00:14<00:00, 19432.27it/s]


In [24]:
print(len(word2idx))

343116


In [16]:
X_train = pad_sequence(X_train, batch_first = True, padding_value=word2idx['<pad>'])
print("X train shape = ", X_train.shape)


X train shape =  torch.Size([288000, 433])


In [17]:
y_train = torch.tensor(y_train)
print("y train shape = ", y_train.shape)
gc.collect()

y train shape =  torch.Size([288000])


41

In [18]:
print(X_train.shape)
print(X_train[0])

torch.Size([288000, 433])
tensor([58642,    85,    17,     3, 84257,    13,    11,   196,   463,    18,
          370,    15,     9,  5688,     3, 68210,    14,    76,   430,    36,
           89,     5,    50,  1626,     9,    85,     8,   134,    79,   606,
        17813,     2,   232,   128,    15,     5,    26,   562,     3,   232,
        34462,  2135,    23,    57,    10,    35,    10,     3,   846,     5,
           26,   124,   562,     9,    55,     3,    94,   128,    15,     9,
         7176,   267,    48,  4120, 58643,     6,   433,     7, 16340,  1166,
           24,  8639,  2957,     6,  4129, 17055,     2,     9,    50,  4814,
          204,    79,  2329,     8,   323,    15, 17544,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,   

In [19]:
print(type(X_train))
print(type(y_train))

<class 'torch.Tensor'>
<class 'torch.Tensor'>


In [20]:
WORD_TO_IDX = os.path.join(root, 'output', 'word2idx.pkl')
IDX_TO_WORD = os.path.join(root, 'output', 'idx2word.pkl')
with open(WORD_TO_IDX, 'wb') as file:
    pickle.dump(word2idx, file)
with open(IDX_TO_WORD, 'wb') as file:
    pickle.dump(idx2word, file)

In [21]:
#save checkpoint after tokens np
X_TOKENS_TRAIN_OUTPUT_PATH = os.path.join(root, 'output', 'X_tokens.npy')
Y_TOKENS_TRAIN_OUTPUT_PATH = os.path.join(root, 'output', 'y_tokens.npy')
with open(X_TOKENS_TRAIN_OUTPUT_PATH, 'wb') as file:
    np.save(file, X_train)
with open(Y_TOKENS_TRAIN_OUTPUT_PATH, 'wb') as file:
    np.save(file, y_train)